In [3]:
%matplotlib inline
import os
import numpy as np

from random import randint
from threading import Thread
from time import sleep, time

from matplotlib import pyplot as plt
from matplotlib import animation
from pandas import read_csv
from IPython.display import HTML
from tqdm.notebook import tqdm

from gwpy.timeseries import TimeSeries
from ts_outlier_detection import *
from ts_outlier_detection.plotting import *
from utils import *

time_header = 'GPStime'
fast_scattering = 'Fast Scattering'
koi_fish = 'Koi Fish'
tomte = 'Tomte'
repeating_blips = 'Repeating Blips'
scattered_light = 'Scattered Light'

h1_fast_scattering_df = read_csv('data/H1_ Fast scattering - Sheet1 (1).csv')
h1_koi_fish_df =        read_csv('data/H1_ Koi Fish - gspy (7) (2).csv')
h1_tomte_df =           read_csv('data/H1_ Tomte - gspy.csv')
h1_repeating_blips_df = read_csv('data/H1_ Repeating_Blips - gspy.csv')
h1_scattered_light_df = read_csv('data/H1_ Scattered light - gspy (1).csv')
l1_fast_scattering_df = read_csv('data/L1_ Fast-scattering - gspy (6) (1).csv')
l1_koi_fish_df =        read_csv('data/L1_ Koi Fish - Sheet1 (1).csv')
l1_tomte_df =           read_csv('data/L1_ Tomte - gspy.csv')
l1_repeating_blips_df = read_csv('data/L1_ Repeating_Blips - gspy.csv')
l1_scattered_light_df = read_csv('data/L1_ Scattered Light - gspy (1) (1).csv')

H1_GLITCHES = {
    fast_scattering: h1_fast_scattering_df[time_header].to_numpy(),
    koi_fish:        h1_koi_fish_df[time_header].to_numpy(),
    scattered_light: h1_scattered_light_df[time_header].to_numpy(),
    tomte:           h1_tomte_df[time_header].to_numpy(),
    repeating_blips: h1_repeating_blips_df[time_header].to_numpy()
}

L1_GLITCHES = {
    fast_scattering: l1_fast_scattering_df[time_header].to_numpy(),
    koi_fish:        l1_koi_fish_df[time_header].to_numpy(),
    scattered_light: l1_scattered_light_df[time_header].to_numpy(),
    tomte:           l1_tomte_df[time_header].to_numpy(),
    repeating_blips: l1_repeating_blips_df[time_header].to_numpy()
}

ALL_GLITCHES = {'H1': H1_GLITCHES, 'L1': L1_GLITCHES}

for det, glitches in ALL_GLITCHES.items():
    for kind, times in glitches.items():
        print(f'{det} has {times.size} {kind} glitches')

CACHE = '/mnt/e/datacache/'

def get_cached_and_uncached_data(name, kwargs, return_failed=False):
    gps_times = []
    for det, glitches in ALL_GLITCHES.items():
        print(f'Fetching {name} events from {det}')
        gps_times.extend([(det, glitch) for glitch in glitches[name]])

    glitch_ts = []
    glitch_uncached_times = []
    glitch_failed_times = []
    cache_path = os.path.join(CACHE, name)

    start = time()
    threads = []
    for det, gps_time in gps_times:
        threads.append(Thread(
            target=insert_cached_data,
            args=(glitch_ts, glitch_uncached_times, det, gps_time, start, cache_path)
        ))
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    threads = []
    for det, gps_time in glitch_uncached_times:
        threads.append(Thread(
            target=insert_event_data,
            args=(glitch_ts, glitch_failed_times, det, gps_time, start, 2, cache_path),
            kwargs=kwargs
        ))
    for t in threads:
        t.start()
        sleep(20)
    for t in threads:
        t.join()
    
    if return_failed:
        return glitch_ts, glitch_failed_times
    return glitch_ts

def insert_event_data(target, failures, det, gps_time, start, retries=2, cache=None, **kwargs):
    success = True
    try:
        ts = get_processed_event(det, gps_time, **kwargs)[0]
        target.append((ts, det, gps_time))
        print(f'Fetched {len(target)} events after {round(time()-start, 2)} seconds')
        
    except Exception as e:
        success = False
        print(f'Error fetching event from {det} at {gps_time}: {e}')
        if retries > 0:
            print(f'Retrying {retries-1} more times')
            sleep(randint(5, 25))
            insert_event_data(target, failures, det, gps_time, start, retries=retries-1, cache=cache)
        else:
            print('No more retries')
            failures.append((det, gps_time))
        
    if success and cache:
        name = '_'.join((det, str(gps_time))) + '.csv'
        pathname = os.path.join(cache, name)
        if not os.path.isdir(cache):
            os.mkdir(cache)
        if not os.path.isfile(pathname):
            ts.write(pathname)

def insert_cached_data(target, failures, det, gps_time, start, cache):
    name = '_'.join((det, str(gps_time))) + '.csv'
    pathname = os.path.join(cache, name)
    if os.path.isfile(pathname):
        target.append((TimeSeries.read(pathname), det, gps_time))
        print(f'Fetched {len(target)} events after {round(time()-start, 2)} seconds')
    else:
        print(f'{pathname} cache file not found')
        failures.append((det, gps_time))

H1 has 999 Fast Scattering glitches
H1 has 100 Koi Fish glitches
H1 has 999 Scattered Light glitches
H1 has 250 Tomte glitches
H1 has 250 Repeating Blips glitches
L1 has 999 Fast Scattering glitches
L1 has 100 Koi Fish glitches
L1 has 999 Scattered Light glitches
L1 has 250 Tomte glitches
L1 has 135 Repeating Blips glitches


In [4]:
## Repeating Blips
get_cached_and_uncached_data(repeating_blips, kwargs={'length': 2, 'bp': [(20, 300)]})

Fetching Repeating Blips events from H1
Fetching Repeating Blips events from L1
/mnt/e/datacache/Repeating Blips/H1_1241577499.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1245663422.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1249399256.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1244879077.csv cache file not found/mnt/e/datacache/Repeating Blips/H1_1244879077.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1241577500.csv cache file not found

/mnt/e/datacache/Repeating Blips/H1_1245852979.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1240932327.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1241393764.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1238174254.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1238770436.csv cache file not found
/mnt/e/datacache/Repeating Blips/H1_1238910335.csv cache file not found/mnt/e/datacache/Repeating Blips/H1_1238716188.csv cache 

Fetched 1 events after 197.58 seconds
Fetched 2 events after 200.08 seconds
Fetched 3 events after 265.86 seconds
Fetched 4 events after 288.29 seconds
Fetched 5 events after 293.55 seconds
Fetched 6 events after 329.21 seconds
Fetched 7 events after 359.37 seconds
Fetched 8 events after 407.82 seconds
Fetched 9 events after 502.51 seconds
Fetched 10 events after 534.45 seconds
Fetched 11 events after 542.34 seconds
Fetched 12 events after 542.37 seconds
Fetched 13 events after 543.78 seconds
Fetched 14 events after 592.06 seconds
Fetched 15 events after 593.53 seconds
Fetched 16 events after 623.36 seconds
Fetched 17 events after 665.8 seconds
Fetched 18 events after 721.77 seconds
Fetched 19 events after 724.75 seconds
Fetched 20 events after 789.79 seconds
Fetched 21 events after 813.49 seconds
Fetched 22 events after 836.37 seconds
Fetched 23 events after 841.16 seconds
Fetched 24 events after 898.83 seconds
Fetched 25 events after 917.66 seconds
Fetched 26 events after 925.59 seco

Fetched 96 events after 3685.48 seconds
Error fetching event from H1 at 1240395285: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1253124570: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1243925407: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1251645264: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1252828663: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1247379276: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1253032606: The read operation timed out
Retrying 1 more times
Fetched 97 events after 3760.16 seconds
Fetched 98 events after 3762.12 seconds
Error fetching event from H1 at 1240558630: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1248590093: The read operation timed out
Retrying 0 more times
Error fetching event from

Error fetching event from H1 at 1246875750: The read operation timed out
No more retries
Error fetching event from H1 at 1241507088: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1243597704: The read operation timed out
No more retries
Error fetching event from H1 at 1244751612: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1249547194: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1245268030: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1239019059: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1244210490: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1248590093: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1241507088: The read operation timed out
No more retries
Error fetching event from H1 at 1244751612: The read operation timed

Error fetching event from H1 at 1252941645: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1252956940: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1238955735: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1240138709: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1242818638: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1249547194: The read operation timed out
No more retries
Error fetching event from H1 at 1238923246: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1243015044: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1253032606: The read operation timed out
No more retries
Error fetching event from H1 at 1241156058: The read operation timed out
No more retries
Error fetching event from H1 at 1253032606: The read operation timed

...

Error fetching event from H1 at 1238423375: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1247617459: The read operation timed out
No more retries
Error fetching event from H1 at 1240138709: The read operation timed out
No more retries
Error fetching event from H1 at 1242182260: The read operation timed out
No more retries
Error fetching event from L1 at 1250046412.784: The read operation timed out
No more retries
Error fetching event from L1 at 1243728862.821: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1244778185: The read operation timed out
Retrying 0 more times
Error fetching event from L1 at 1242412416.868: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1245259159.012: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1238423375: The read operation timed out
No more retries
Error fetching event from L1 at 1251552965.981: The read operati

Error fetching event from L1 at 1253354587.476: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1246107818.268: The read operation timed out
No more retries
Error fetching event from H1 at 1244000219: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1251270743.279: The read operation timed out
Retrying 0 more times
Error fetching event from L1 at 1249316775.527: The read operation timed out
No more retries
Error fetching event from L1 at 1248757443.993: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1251204912.204: The read operation timed out
Retrying 0 more times
Error fetching event from L1 at 1242412416.868: The read operation timed out
No more retries
Error fetching event from L1 at 1253354587.476: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1245684312: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1242

Error fetching event from L1 at 1248317928.911: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying 0 more times
Error fetching event from L1 at 1252535302.624: The read operation timed out
No more retries
Error fetching event from H1 at 1252956940: The read operation timed out
No more retries
Error fetching event from L1 at 1252067755.181: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1246376468.562: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1241745574: The read operation timed out
No more retries
Error fetching event from L1 at 1250389871.736: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1244030330: The read operation timed out
Retrying 0 more times
Error fetching event from L1 at 1246478966.571: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1241297375.756: The read operation timed ou

Error fetching event from L1 at 1253505377.63: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1247773537.499: The read operation timed out
Retrying 1 more times
Error fetching event from L1 at 1241454559.766: The read operation timed out
Retrying 1 more times
Error fetching event from H1 at 1245721533: The read operation timed out
Retrying 0 more times
Error fetching event from L1 at 1242481931.376: The read operation timed out
Retrying 0 more times
Error fetching event from H1 at 1239684612: The read operation timed out
No more retries
Error fetching event from H1 at 1244713445: The read operation timed out
No more retries
Error fetching event from L1 at 1250380395.621: The read operation timed out
Retrying 0 more times
Error fetching event from L1 at 1248757443.993: The read operation timed out
No more retries
Error fetching event from L1 at 1250619711.819: The read operation timed out
Retrying 0 more times
Error fetching event from L1 at 124279194

[(<TimeSeries([0.21771622, 0.21467086, 0.1852744 , ..., 0.32269323,
               0.3351694 , 0.31494711]
              unit=Unit(dimensionless),
              t0=<Quantity 1.2415775e+09 s>,
              dt=<Quantity 0.00024414 s>,
              name='Strain',
              channel=None)>,
  'H1',
  1241577499),
 (<TimeSeries([-0.11256718, -0.09356501, -0.06852541, ...,
                0.3627418 ,  0.39409917,  0.39279569]
              unit=Unit(dimensionless),
              t0=<Quantity 1.24566342e+09 s>,
              dt=<Quantity 0.00024414 s>,
              name='Strain',
              channel=None)>,
  'H1',
  1245663422),
 (<TimeSeries([ 0.17581499,  0.27701369,  0.36128317, ...,
               -0.24374192, -0.26070352, -0.23346106]
              unit=Unit(dimensionless),
              t0=<Quantity 1.24939925e+09 s>,
              dt=<Quantity 0.00024414 s>,
              name='Strain',
              channel=None)>,
  'H1',
  1249399256),
 (<TimeSeries([0.10505065, 0.12405812,